In [16]:
import math
import random
import numpy as np
import pandas as pd

In [1]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
    def copy(self):
        new_instance = Node(self.value)
        if self.right is not None:
            new_instance.right = self.right.copy()
        if self.left is not None:
            new_instance.left = self.left.copy()
        return new_instance

In [2]:
def evaluate_tree(tree, values):

    if tree == log:
        return np.log(np.abs(values['x']))
    if tree == func3:
        return func3(values)

    if tree.value == '+':
        return evaluate_tree(tree.left, values) + evaluate_tree(tree.right, values)
    elif tree.value == '-':
        return evaluate_tree(tree.left, values) - evaluate_tree(tree.right, values)
    elif tree.value == '*':
        return evaluate_tree(tree.left, values) * evaluate_tree(tree.right, values)
    if tree.value == '^':
        return evaluate_tree(tree.left, values) ** evaluate_tree(tree.right, values)
    elif tree.value == '/':
        right_val = evaluate_tree(tree.right, values)
        # Handle division by zero with numpy where division by zero can result in np.inf
        return np.divide(evaluate_tree(tree.left, values), right_val, where=(right_val!=0), out=np.full_like(right_val, 10000))
    elif tree.value == 'cos':
        return np.cos(evaluate_tree(tree.left, values))
    elif tree.value == 'sin':
        return np.sin(evaluate_tree(tree.left, values))
    
    else: # Terminal node
        if isinstance(tree.value, float):
            return tree.value
        else:
            return np.array(values[tree.value], dtype=np.float64)

In [3]:
def evaluate_fitness(tree, data_points, target_tree):
    errors = np.zeros(len(data_points))
    for i, x in data_points.iterrows():
        errors[i] = (evaluate_tree(tree, x) - evaluate_tree(target_tree, x))**2
    return -np.sqrt(np.sum(errors))

In [4]:
def generate_dataset(num_points, minn, maxx, num_input):
    data = {}
    data['x'] = np.random.uniform(minn, maxx, num_points)
    if num_input > 1:
        data['y'] = np.random.uniform(minn, maxx, num_points)
    if num_input > 2:
        data['z'] = np.random.uniform(minn, maxx, num_points)
    return pd.DataFrame(data)

In [5]:
def crossover(par1, par2):
    parent1 = par1.copy()
    parent2 = par2.copy()
    # crossover_point = random.randint(1, 2)
    # if crossover_point == 1:
    parent1.left, parent2.left = parent2.left, parent1.left
    # else:
        # parent1.right, parent2.right = parent2.right, parent1.right
    return parent1, parent2

In [6]:
def mutation(tree, num_input):
    vars = ['x', 'y', 'z']
    mutate_node = tree
    i = 0
    tmp = random.randint(1, 7)
    while (mutate_node.left is not None or mutate_node.right is not None) and i < tmp:
        i+=1
        if mutate_node.left is not None and mutate_node.right is not None:
            mutate_node = random.choice([mutate_node.left, mutate_node.right])
        elif mutate_node.left is not None:
            mutate_node = mutate_node.left
        else:
            mutate_node = mutate_node.right
    
    mutate_node.value = random.choice(['+', '-', '*', '/','+', '-', '*', '/','cos', 'sin'])

    if mutate_node.value in ['sin', 'cos']:
        mutate_node.left = Node(random.choice(random.choice(vars[:num_input])))
        return tree

    r = random.randint(1, 4)
    if r ==1:
        mutate_node.right = Node(random.choice(random.choice(vars[:num_input])))
        mutate_node.left = Node(random.choice(random.choice(vars[:num_input])))
    elif r==2:
        mutate_node.right = Node(random.choice(random.choice(vars[:num_input])))
        mutate_node.left = Node(float(random.randint(0,10)))
    elif r==3:
        mutate_node.right = Node(float(random.randint(0,10)))
        mutate_node.left = Node(random.choice(random.choice(vars[:num_input])))
    else:
        mutate_node.right = Node(float(random.randint(0,10)))
        mutate_node.left = Node(float(random.randint(0,10)))

    return tree

In [7]:
def initialize_population(population_size, num_input):
    population = []
    vars = ['x', 'y', 'z']
    for _ in range(population_size):
        tree = Node(random.choice(['+', '-', '*', '/', 'sin', 'cos']))
        tree.left = Node(random.choice(vars[:num_input]))  # Terminal
        tree.right = Node(random.choice(vars[:num_input]))  # Terminal
        population.append(tree)
    return population

In [9]:
def GP(data_points, population_size, num_generation, target_tree, num_input):
    population = initialize_population(population_size, num_input)
    for i in range(num_generation):
        print("here")
        new_generation = []
        for __ in range(int(population_size / 2)):
            fitness_scores = [evaluate_fitness(individual, data_points, target_tree) for individual in population]

            if max(fitness_scores) >= -1:
                max_score_index = fitness_scores.index(max(fitness_scores))
                best_chromosome = population[max_score_index]
                return best_chromosome

            minn = min(fitness_scores)
            positive_fitness_scores = [score + minn for score in fitness_scores]

            total_fitness = sum(positive_fitness_scores)
            normalized_fitness = [100 * score / total_fitness for score in positive_fitness_scores]

            rand_val = random.randint(1, 100)
            cumulative_prob = 0
            flg = False
            for _ in range(population_size):
                if cumulative_prob >= rand_val:
                    par1 = population[_]
                    flg = True
                    break
                cumulative_prob += normalized_fitness[_]

            if not flg:
                par1 = population[-1]

            rand_val = random.randint(1, 100)
            cumulative_prob = 0
            flg = False
            for _ in range(population_size):
                if cumulative_prob >= rand_val:
                    par2 = population[_]
                    flg = True
                    break
                cumulative_prob += normalized_fitness[_]
            if not flg:
                par2 = population[-1]

            child1, child2 = crossover(par1, par2)
            ev1 = evaluate_fitness(child1, data_points, target_tree)
            ev2 = evaluate_fitness(child2, data_points, target_tree)
            child = child2 if ev1 < ev2 else child1

            prob = random.randint(0, 100)
            if prob < 25:
                child = mutation(child, num_input)
            new_generation.append(child)

        new_generation_fitness = [evaluate_fitness(individual, data_points, target_tree) for individual in new_generation]

        combined_population = population + new_generation
        combined_fitness = fitness_scores + new_generation_fitness
        combined_data = list(zip(combined_population, combined_fitness))
        sorted_data = sorted(combined_data, key=lambda x: x[1], reverse=True)

        top_individuals = sorted_data[:population_size]
        population, top_fitness = zip(*top_individuals)
        population = list(population)

    max_score_index = fitness_scores.index(max(fitness_scores))
    best_chromosome = population[max_score_index]
    return best_chromosome

In [10]:
# Print infix expression for visualization
def print_infix_expression(node):
    if node is None:
        return ""

    if node.value in {'+', '-', '*', '/'}:
        left_expr = print_infix_expression(node.left)
        right_expr = print_infix_expression(node.right)
        return "(" + left_expr + " " + node.value + " " + right_expr + ")"

    elif node.value in {'sin', 'cos'}:
        return node.value + "(" + print_infix_expression(node.left) + ")"

    return str(node.value)

In [11]:
root = Node('+')
root.right = Node('x')
root.left = Node('y')

vals = {'x':5, 'y':8, 'z':6}

print_infix_expression(root)

'(y + x)'

In [17]:
evaluate_tree(root, vals)

np.float64(13.0)

In [18]:
mutation(root, 2)
print(print_infix_expression(root))
print(evaluate_tree(root, vals))

((x * 5.0) + x)
30.0


In [14]:
def linear():
    root = Node('+')
    mult = Node('*')
    root.left = mult
    root.right = Node(5.0)
    mult.left = Node(3.0)
    mult.right = Node('x')

    return root

def func1():
    plus_node = Node("+")
    plus_node2 = Node("+")
    plus_node3 = Node("+")
    mult_node = Node("*")
    div_node = Node("/")
    minus_node = Node("-")
    x_node = Node("x")
    y_node = Node("y")
    three_node = Node(3.0)
    five_node = Node(5.0)
    const_node = Node(3.14)
    two_node = Node(2.0)

    plus_node.left = mult_node
    plus_node.right = minus_node

    mult_node.left = two_node
    mult_node.right = const_node

    minus_node.left = plus_node2
    minus_node.right = div_node

    plus_node2.left = x_node
    plus_node2.right = three_node

    div_node.left = y_node
    div_node.right = plus_node3

    plus_node3.right = five_node
    plus_node3.left = Node(1.0)

    return plus_node

def func2():
    x_node = Node('x')
    y_node = Node('y')
    two_node = Node(2.0)

    multiply_node = Node('*')
    subtract_node = Node('-')

    subtract_node.left = x_node
    subtract_node.right = multiply_node

    multiply_node.left = two_node
    multiply_node.right = y_node

    root = subtract_node
    return root

def log(values):
    return math.log(abs(values['x']))

def func3(values):
    if values['x'] <= 0:
        return 2**values['y']
    if values['x'] <= 50:
        return 3 - values['y']
    return math.sin(values['y'])

def func4():
    # sin_node = Node('sin')
    # plus_node = Node('+')
    # mult_node = Node('*')
    # minus_node = Node('-')
    # mult_node2 = Node('*')

    # plus_node.left = sin_node
    # plus_node.right = mult_node

    # sin_node.left = Node('x')

    # mult_node.left = Node(10.0)
    # mult_node.right = minus_node

    # minus_node.left = Node('y')
    # minus_node.right = mult_node2

    # mult_node2.left = Node(5.0)
    # mult_node2.right = Node('z')



    div_node = Node('/')
    mult_node = Node('*')
    minus_node = Node('-')
    
    div_node.right = Node('z')
    div_node.left = Node('x')

    mult_node.right = Node('y')
    mult_node.left = Node(2.0)

    minus_node.right = div_node
    minus_node.left = mult_node


    return minus_node
    

In [225]:
def generate_dataset(num_points, minn, maxx, num_input):
    X = []
    for _ in range(num_points):
        point = {}
        point['x']= random.uniform(minn, maxx)
        if num_input >1:
            point['y']= random.uniform(minn, maxx)
        if num_input>2:
            point['z']= random.uniform(minn, maxx)
        X.append(point)
    return X

In [19]:
linear_data = generate_dataset(100, -100, 100, 1)
tree = GP(linear_data, 50, 100, linear(), 1)
print(print_infix_expression(tree))
print(evaluate_fitness(tree, linear_data, linear()))

here


KeyboardInterrupt: 

In [228]:
func1_data = generate_dataset(100, -100, 100, 2)
tree = GP(func1_data, 30, 150, func1(), 2) 
print(print_infix_expression(tree))
print(evaluate_fitness(tree, func1_data, func1()))

(x + (9.0 - (y / 4.0)))
-51.2874470014596


In [229]:
log_data = generate_dataset(100, -100, 100, 1)
tree = GP(log_data, 30, 150, log, 1)
print(print_infix_expression(tree))
print(evaluate_fitness(tree, log_data, log))

((((x - 1.0) + x) + ((x - 1.0) + x)) / x)
-9.805309671568008


In [231]:
func3_data = generate_dataset(100, -500, 500, 2)
tree = GP(func3_data, 50, 200, func3, 2)
print(print_infix_expression(tree))
print(evaluate_fitness(tree, func3_data, func3))

cos(x)
-2.2838550695761106e+149


In [238]:
func4_data = generate_dataset(100, -100, 100, 3)
tree = GP(func4_data, 50, 250, func4(), 3)
print(print_infix_expression(tree))
print(evaluate_fitness(tree, func4_data, func4()))

(y + (y - (x / z)))
-8.377175307253705e-14
